# CT ROI Analysis
## Moving back to basics - look at all of the CTs to generate ROIs for the PET images

In [1]:
import os
import pandas as pd
import nibabel as nib
import numpy as np
from pyanalyze.functions import *

PET_CT_REF = pd.read_excel("D:/Hoffman/DCM_Partners.xlsx")
PET_CT_REF["Centre"] = [x.split("\\")[2][1:] for x in PET_CT_REF["PET"]]

centrecount, prevcentre,seriesnumlist = 1, "",[]
for index,row in PET_CT_REF.iterrows():
    if row["Centre"] != prevcentre:
        prevcentre = row["Centre"]
        centrecount = 1
        seriesnumlist.append(str(centrecount).zfill(2))
    else:
        seriesnumlist.append(str(centrecount).zfill(2))
    centrecount+=1
PET_CT_REF["SeriesNum"] = seriesnumlist

pcentre, pct, count,seriesnumlist = "", "",1,[]
for index,row in PET_CT_REF.iterrows():
    if row["Centre"] != pcentre:
        pcentre = row["Centre"]
        pct = row["CT"]
        count = 1
    elif row["Centre"] == pcentre and row["CT"] != pct:
        pct = row["CT"]
        count += 1
    seriesnumlist.append(str(count).zfill(2))
PET_CT_REF["CT_SeriesNum"] = seriesnumlist

PET_CT_REF.tail()

In [ ]:
PET_CT_REF["PixelSpacing"] = [get_dcm_pixel_spacing(x) for x in PET_CT_REF["PET"]]
PET_CT_REF["MatrixSize"] = [get_dcm_matrix_size(x) for x in PET_CT_REF["PET"]]
PET_CT_REF["Iterations_Subsets"] = [get_iterations_subsets(x) for x in PET_CT_REF["PET"]]
PET_CT_REF.head()

,PET,CT,Centre,SeriesNum,CT_SeriesNum,PixelSpacing,MatrixSize,Iterations_Subsets
0,D:\Hoffman\_Aberdeen\IMANOVA_BRAIN+SKULL\ARI P...,D:\Hoffman\CT_AllCentres\_Aberdeen\Skull\WB St...,Aberdeen,01,01,"(0.9765625, 0.9765625, 3.2700)","(256, 256, 47)","(nan, nan)"
1,D:\Hoffman\_Aberdeen\IMANOVA_BRAIN+SKULL\ARI P...,D:\Hoffman\CT_AllCentres\_Aberdeen\Skull\WB St...,Aberdeen,02,01,"(0.9765625, 0.9765625, 3.2700)","(256, 256, 47)","(nan, nan)"
2,D:\Hoffman\_Aberdeen\IMANOVA_BRAIN+SKULL\ARI P...,D:\Hoffman\CT_AllCentres\_Aberdeen\Skull\WB St...,Aberdeen,03,01,"(0.9765625, 0.9765625, 3.2700)","(256, 256, 47)","(nan, nan)"
3,D:\Hoffman\_Aberdeen\IMANOVA_BRAIN+SKULL\ARI P...,D:\Hoffman\CT_AllCentres\_Aberdeen\Skull\WB St...,Aberdeen,04,01,"(0.9765625, 0.9765625, 3.2700)","(256, 256, 47)","(nan, nan)"
4,D:\Hoffman\_Aberdeen\IMANOVA_BRAIN+SKULL\ARI P...,D:\Hoffman\CT_AllCentres\_Aberdeen\Skull\WB St...,Aberdeen,05,01,"(0.9765625, 0.9765625, 3.2700)","(256, 256, 47)","(nan, nan)"


### Add the activities and other references - following cells taken from Book1

In [ ]:
activities_sheet = pd.read_excel("D:/Hoffman/GN/image_reference.xlsx", sheet_name="Activities")
activities_sheet.drop(columns=["Notes"],inplace=True)
image_ref_sheet = pd.read_excel("D:/Hoffman/GN/image_reference.xlsx", sheet_name="DCM_reference")
image_ref_sheet.head()

,Path,PET/CT,Contains 6 Frames?,5m PET,Manufacturer,Scanner,Skull,Activity,Standardised
0,D:/Hoffman/_Aberdeen/IMANOVA_BRAIN_NO_SKULL/AR...,P,True,True,GE,D710,False,NaN,0.0
1,D:/Hoffman/_Aberdeen/IMANOVA_BRAIN_NO_SKULL/IM...,P,True,True,GE,D710,False,NaN,1.0
2,D:/Hoffman/_Aberdeen/IMANOVA_BRAIN_NO_SKULL/WB...,C,False,False,GE,D710,False,NaN,0.0
3,D:/Hoffman/_Aberdeen/IMANOVA_BRAIN+SKULL/ARI P...,P,True,True,GE,D710,True,NaN,0.0
4,D:/Hoffman/_Aberdeen/IMANOVA_BRAIN+SKULL/IMANO...,P,True,True,GE,D710,True,NaN,1.0


In [ ]:
image_ref_index = []
for index,row in PET_CT_REF.iterrows():
    for index2,row2 in image_ref_sheet.iterrows():
        if any([os.path.abspath(row["PET"]) == os.path.join(row2["Path"],"Frame{}".format(i)) for i in range(1,7)]):
            image_ref_index.append(index2)
            break
PET_CT_REF["ImageRefIndex"] = image_ref_index

ValueError: Length of values (0) does not match length of index (463)